<a href="https://colab.research.google.com/github/DepartmentOfStatisticsPUE/ann-for-survey-sampling/blob/main/notebooks/ann_paper_sim_study_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install required packages for the simulation study

In [ ]:
!pip install annoy

In [ ]:
!pip install pynndescent

In [ ]:
!pip install hnswlib

Load standard modules

In [1]:
import pandas as pd
import numpy as np
import plotnine as ggplot

Load ann modules

In [ ]:
from sklearn.neighbors import KDTree ## exact search
from annoy import AnnoyIndex

Generate data for the simulation study

In [6]:
np.random.seed(123)
N = 1000000
x1 = np.random.normal(loc=1.0,scale=1.0,size=N)
x2 = np.random.exponential(scale=1.0, size = N)
epsilon = np.random.normal(size=N)

### target variables
y1 = 1 + x1 + x2 + epsilon
y2 = 0.5*(x1 - 1.5)**2 + x2 + epsilon
## propensity scores
p1 = np.exp(x2) / (1 + np.exp(x2))
p2 = np.exp(-0.5 + 0.5*(x2-2)**2) / (1 + np.exp(-0.5 + 0.5*(x2-2)**2))

data = np.column_stack((x1,x2,y1,y2,p1,p2))
data[:5]

array([[-0.0856306 ,  0.55466463,  0.90054107,  1.24328388,  0.63521714,
         0.63285795],
       [ 1.99734545,  1.19758235,  4.45522212,  1.58155292,  0.76809442,
         0.45560151],
       [ 1.2829785 ,  0.84342942,  3.025612  ,  0.76618266,  0.699187  ,
         0.54210697],
       [-0.50629471,  4.28335756,  4.09344198,  5.61234593,  0.98639148,
         0.89156823],
       [ 0.42139975,  1.56705153,  2.21403432,  1.37432383,  0.82736287,
         0.39980497]])

Simulation study

In [ ]:
R = 2000

s500 = np.random.choice(a = data.shape[0], size = 500, replace = False)
s1000 = np.random.choice(a = data.shape[0], size = 500, replace = False)